In [2]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value

class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:   #不管是什麼錯誤
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()
# model_dump 將對象轉化為Dict，之後就可以調用python標準庫序列化json字串，会序列化嵌套对象
# 也可以使用dict(model)将对象转化为字典，但嵌套对象不会被转化为字典。
df = pd.DataFrame(all_sites)
df

,行政區域,總數量,可借數量,可還數量,時間
0,大安區,28,10,18,2024-07-06 14:48:18
1,大安區,21,8,13,2024-07-06 14:39:20
2,大安區,16,5,11,2024-07-06 14:41:19
3,大安區,11,2,8,2024-07-06 14:47:19
4,大安區,16,7,8,2024-07-06 14:38:20
...,...,...,...,...,...
1425,臺大公館校區,30,30,0,2024-07-06 14:48:14
1426,臺大公館校區,20,9,11,2024-07-06 14:48:18
1427,臺大公館校區,24,17,7,2024-07-06 14:43:19
1428,臺大公館校區,40,22,18,2024-07-06 14:48:19


In [3]:
# 使用by  ，會自動將by的欄位 變成索引
# [[選取我所要統計的columns]]
df.groupby(by='行政區域')[['總數量','可借數量','可還數量']].sum()

# 跟20_0 最後一個 level=行政區域 一樣

,總數量,可借數量,可還數量
行政區域,,,
中山區,3819,1426,2364
中正區,3224,1374,1807
信義區,3331,1371,1919
內湖區,3499,1451,2031
北投區,2472,1075,1356
南港區,2317,1080,1228
士林區,2962,1246,1651
大同區,1629,715,885
大安區,4987,2055,2865


groupby實體.統計function()  
若要使用多個function：.agg(   ex:count(),sum()  )

In [4]:
df.groupby(by='行政區域')[['總數量','可借數量','可還數量']].agg(['sum','count'])

總數量        可借數量        可還數量      
         sum count   sum count   sum count
行政區域                                      
中山區     3819   156  1426   156  2364   156
中正區     3224   121  1374   121  1807   121
信義區     3331   106  1371   106  1919   106
內湖區     3499   168  1451   168  2031   168
北投區     2472    94  1075    94  1356    94
南港區     2317    92  1080    92  1228    92
士林區     2962   128  1246   128  1651   128
大同區     1629    62   715    62   885    62
大安區     4987   185  2055   185  2865   185
文山區     2203    97  1048    97  1152    97
松山區     2343    88  1009    88  1319    88
臺大公館校區  1437    53   544    53   882    53
萬華區     2057    80   877    80  1075    80

In [5]:
# 多個統計function，使用 agg  list裡面放置tuple
df1 = df.groupby(by='行政區域')[['總數量','可借數量','可還數量']].agg([('車輛數','sum'),('站點數','count')])
df1

總數量       可借數量       可還數量     
         車輛數  站點數   車輛數  站點數   車輛數  站點數
行政區域                                   
中山區     3819  156  1426  156  2364  156
中正區     3224  121  1374  121  1807  121
信義區     3331  106  1371  106  1919  106
內湖區     3499  168  1451  168  2031  168
北投區     2472   94  1075   94  1356   94
南港區     2317   92  1080   92  1228   92
士林區     2962  128  1246  128  1651  128
大同區     1629   62   715   62   885   62
大安區     4987  185  2055  185  2865  185
文山區     2203   97  1048   97  1152   97
松山區     2343   88  1009   88  1319   88
臺大公館校區  1437   53   544   53   882   53
萬華區     2057   80   877   80  1075   80

#### stacking 和 unstacking 被廣泛用於改變正在考慮的 DataFrame 的形狀。 使行變成列，列相應地變成行。
DataFrame.stack(level=-1, dropna=_NoDefault.no_default, sort=_NoDefault.no_default, future_stack=False)

- 1.stack：將數據的列“旋轉”為行    (橫轉直)
- 2.unstack：將數據的行“旋轉”為列  (直轉橫)
- 3.stack和unstack默認操作為最內層
- 4.stack和unstack默認旋轉軸的級別將會成果結果中的最低級別（最內層）
- 5.stack和unstack為一組逆運算操作

In [6]:
#pivot 樞紐分析
#stack,unstack
#欄位有名稱
s1 = df1.stack(level=[0,1])
s1

C:\Users\Han PC\AppData\Local\Temp\ipykernel_23288\2724416305.py:4: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  s1 = df1.stack(level=[0,1])


行政區域           
中山區   可借數量  車輛數    1426
            站點數     156
      可還數量  車輛數    2364
            站點數     156
      總數量   車輛數    3819
                   ... 
萬華區   可借數量  站點數      80
      可還數量  車輛數    1075
            站點數      80
      總數量   車輛數    2057
            站點數      80
Length: 78, dtype: int64

In [7]:
s1.index.names = ['行政區域','主題','數量']
s1

行政區域  主題    數量 
中山區   可借數量  車輛數    1426
            站點數     156
      可還數量  車輛數    2364
            站點數     156
      總數量   車輛數    3819
                   ... 
萬華區   可借數量  站點數      80
      可還數量  車輛數    1075
            站點數      80
      總數量   車輛數    2057
            站點數      80
Length: 78, dtype: int64

In [8]:
# 設為 column
s1.unstack(level=['數量'])

數量            車輛數  站點數
行政區域   主題             
中山區    可借數量  1426  156
       可還數量  2364  156
       總數量   3819  156
中正區    可借數量  1374  121
       可還數量  1807  121
       總數量   3224  121
信義區    可借數量  1371  106
       可還數量  1919  106
       總數量   3331  106
內湖區    可借數量  1451  168
       可還數量  2031  168
       總數量   3499  168
北投區    可借數量  1075   94
       可還數量  1356   94
       總數量   2472   94
南港區    可借數量  1080   92
       可還數量  1228   92
       總數量   2317   92
士林區    可借數量  1246  128
       可還數量  1651  128
       總數量   2962  128
大同區    可借數量   715   62
       可還數量   885   62
       總數量   1629   62
大安區    可借數量  2055  185
       可還數量  2865  185
       總數量   4987  185
文山區    可借數量  1048   97
       可還數量  1152   97
       總數量   2203   97
松山區    可借數量  1009   88
       可還數量  1319   88
       總數量   2343   88
臺大公館校區 可借數量   544   53
       可還數量   882   53
       總數量   1437   53
萬華區    可借數量   877   80
       可還數量  1075   80
       總數量   2057   80

In [9]:
s1.unstack(level=['主題'])

主題          可借數量  可還數量   總數量
行政區域   數量                   
中山區    車輛數  1426  2364  3819
       站點數   156   156   156
中正區    車輛數  1374  1807  3224
       站點數   121   121   121
信義區    車輛數  1371  1919  3331
       站點數   106   106   106
內湖區    車輛數  1451  2031  3499
       站點數   168   168   168
北投區    車輛數  1075  1356  2472
       站點數    94    94    94
南港區    車輛數  1080  1228  2317
       站點數    92    92    92
士林區    車輛數  1246  1651  2962
       站點數   128   128   128
大同區    車輛數   715   885  1629
       站點數    62    62    62
大安區    車輛數  2055  2865  4987
       站點數   185   185   185
文山區    車輛數  1048  1152  2203
       站點數    97    97    97
松山區    車輛數  1009  1319  2343
       站點數    88    88    88
臺大公館校區 車輛數   544   882  1437
       站點數    53    53    53
萬華區    車輛數   877  1075  2057
       站點數    80    80    80

In [10]:
import pandas as pd

# Sample DataFrame
df = pd.DataFrame({
    'A': ['one', 'two', 'three'],
    'B': ['four', 'five', 'six'],
    'C': [1, 2, 3]
})
print(df)

       A     B  C
0    one  four  1
1    two  five  2
2  three   six  3


In [11]:
stacked_df = df.stack()
print(stacked_df)

0  A      one
   B     four
   C        1
1  A      two
   B     five
   C        2
2  A    three
   B      six
   C        3
dtype: object


In [12]:
unstacked_df = stacked_df.unstack()
print(unstacked_df)

       A     B  C
0    one  four  1
1    two  five  2
2  three   six  3


In [13]:
partially_stacked = df[['A', 'B']].stack()
print(partially_stacked)

0  A      one
   B     four
1  A      two
   B     five
2  A    three
   B      six
dtype: object


In [14]:
multi_level_df = df.stack()
unstacked_by_level = multi_level_df.unstack(level=0)
print(unstacked_by_level)

      0     1      2
A   one   two  three
B  four  five    six
C     1     2      3


In [15]:
df_with_na = pd.DataFrame({
    'A': ['one', None, 'three'],
    'B': ['four', 'five', 'six'],
    'C': [1, 2, 3]
})
stacked_with_na = df_with_na.stack()
print(df_with_na)
print(stacked_with_na)
# 自動排除缺失值，從而簡化資料清理過程。

       A     B  C
0    one  four  1
1   None  five  2
2  three   six  3
0  A      one
   B     four
   C        1
1  B     five
   C        2
2  A    three
   B      six
   C        3
dtype: object


In [16]:
multi_col_df = pd.DataFrame({
    ('A', 'cat'): ['one', 'two', 'three'],
    ('B', 'dog'): ['four', 'five', 'six'],
    ('C', 'mouse'): [1, 2, 3]
}).set_index([('A', 'cat')]).stack()
print(multi_col_df)

                   B    C
(A, cat)                 
one      dog    four  NaN
         mouse   NaN  1.0
two      dog    five  NaN
         mouse   NaN  2.0
three    dog     six  NaN
         mouse   NaN  3.0


C:\Users\Han PC\AppData\Local\Temp\ipykernel_23288\3534490158.py:5: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  }).set_index([('A', 'cat')]).stack()
